In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ["alt.atheism", "soc.religion.christian", "comp.graphics", "sci.med"]

news = fetch_20newsgroups(categories=categories, shuffle=False, random_state=42)

X= news.data
y = news.target

In [3]:
X[0]

"From: keith@cco.caltech.edu (Keith Allan Schneider)\nSubject: Re: <Political Atheists?\nOrganization: California Institute of Technology, Pasadena\nLines: 14\nNNTP-Posting-Host: lloyd.caltech.edu\n\nbobbe@vice.ICO.TEK.COM (Robert Beauchaine) writes:\n\n>To show that the examples I and others\n>have provided are *not* counter examples of your supposed inherent\n>moral hypothesis, you have to successfully argue that\n>domestication removes or alters this morality.\n\nI think that domestication will change behavior to a large degree.\nDomesticated animals exhibit behaviors not found in the wild.  I\ndon't think that they can be viewed as good representatives of the\nwild animal kingdom, since they have been bred for thousands of years\nto produce certain behaviors, etc.\n\nkeith\n"

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=1)

In [8]:
import re
def preprocessor(text):
    text = re.sub("<[^>]*>","", text)
    text = re.sub("[\W]+"," ",text)
    text = text.lower()
    return text

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    text_tokens = nltk.word_tokenize(text)
    return [porter.stem(word) for word in text_tokens]

from nltk.corpus import stopwords
import nltk
nltk.download("punkt")
nltk.download("stopwords")

stop = stopwords.words("english")
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wlska\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wlska\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=preprocessor, tokenizer=tokenizer_porter, stop_words=stop, max_df=0.1, min_df=10)
X_train_vector = tfidf.fit_transform(X_train)
X_test_vector = tfidf.transform(X_test)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [13]:
X_train_vector = X_train_vector.toarray()
X_test_vector = X_test_vector.toarray()
X_train_vector.shape

(1579, 2796)

In [14]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty="l2", verbose=1)
lr.fit(X_train_vector, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished


LogisticRegression(verbose=1)

In [15]:
lr.score(X_train_vector, y_train)

0.9911336288790373

In [16]:
lr.score(X_test_vector, y_test)

0.9542772861356932

In [24]:
docs = ['The outbreak was declared a global pandemic by the World Health Organization (WHO) on 11 March.',

'Today, computer graphics is a core technology in digitalphotography, film, video games, cell phone and computer displays,and many specialized applications.',

'Arguments for atheism range from philosophical to social and historical approaches.',

'The Bible is a compilation of many shorter books written at different times by a variety of authors, and later assembled into the biblical canon.'
]

test = tfidf.transform(docs)
lr.predict(test)

array([2, 1, 0, 3], dtype=int64)

In [25]:
news.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [18]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train_vector, y_train)

DecisionTreeClassifier()

In [19]:
tree.score(X_train_vector, y_train)

1.0

In [22]:
tree.score(X_test_vector, y_test)

0.8362831858407079

In [26]:
tree.predict(test)

array([3, 1, 0, 2], dtype=int64)